In [83]:
import tweepy
from tweepy import OAuthHandler

consumer_key = 'Hw1ILZjbkp5s4SEjwLIgyokgK'
consumer_secret = '3Uw0mtBon6mQkLjJOLLbFxcw2L02MCVKzVeqMfPhYnPLGNkc4e'
access_token = '1108097257092628486-M2GJEh8WydUUI7N78fkBaEYsTZf5O8'
access_secret = 'C2I9EZfsYpUlzVNKzKjNEvonjZdk4Z5eDXmmELC2rhzKH'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [84]:
# Print the last 10 tweets from my own twitter timeline @SamuelPRandall
for status in tweepy.Cursor(api.home_timeline).items(10):
    print(status.text)

I only hang out with the hottest dudes
I'm making these Tweets to test the Twitter API.
This is an even longer Tweet
This is a longer Tweet
This is a Tweet
What is wrong with you people
At least my hairdresser thinks I'm cute
I miss when I got A's on exams most of all damn
I miss Paul Saadeh
I miss Julian Castillo


In [85]:
# Example uses of user data
for friend in tweepy.Cursor(api.friends).items(10):
    print(friend.name)
    print(friend.screen_name)
    print(friend.location)
    print(friend.followers_count)

Sammy Hagar
SandallRamuel
Austin, TX
88


In [86]:
def process_or_store(tweet):
    print(json.dumps(tweet))

In [87]:
from tweepy import Stream
from tweepy.streaming import StreamListener

class MyListener(StreamListener):
    
    def on_data(self,data):
        try:
            with open('python.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on data: %s" % str(e))
            return True
        
    def on_error(self,status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#python'])

KeyboardInterrupt: 

In [88]:
import json
 
with open('python.json', 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4)) # pretty-print


{
    "created_at": "Tue Mar 19 20:48:28 +0000 2019",
    "id": 1108107994250665985,
    "id_str": "1108107994250665985",
    "text": "\u2605\u5145\u5b9f\u2605\u798f\u5229\u539a\u751f\u306f\u5145\u5b9f\u3057\u3066\u3044\u308b\u306e\u3067\u3068\u3066\u3082\u50cd\u304d\u3084\u3059\u3044\u74b0\u5883\u3002\u6709\u7d66\u4f11\u6687\u3001\u793e\u4f1a\u4fdd\u967a\u3001\u539a\u751f\u5e74\u91d1\u5b8c\u5099\nhttps://t.co/o5jCZ7xcEn\nIndeed \u63b2\u8f09\u4e2d\u3067\u3059\n#Python #Unity #AWS #OpenGL\u30a2\u30d7\u30ea #\u958b\u767a #sbw #\u4e09\u548c\u4e8b\u52d9\u5668",
    "source": "<a href=\"http://twittbot.net/\" rel=\"nofollow\">twittbot.net</a>",
    "truncated": false,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 1065026987637399552,
        "id_str": "1065026987637399552",
        "name": "\u3010\u52df\u96c6\u3011\u30b7\u30b9\u30

In [ ]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [62]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']


In [63]:
terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]

In [64]:
# Count terms only once, equivalent to Document Frequency
terms_single = set(terms_all)
# Count hashtags only
terms_hash = [term for term in preprocess(tweet['text']) 
              if term.startswith('#')]
# Count terms only (no hashtags, no mentions)
terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 
              # mind the ((double brackets))
              # startswith() takes a tuple (not a list) if 
              # we pass a list of inputs


In [72]:
from nltk import bigrams 
 
terms_bigram = bigrams(terms_stop)

In [89]:
import operator 
import json
from collections import Counter
 
fname = 'python.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'])]
        # Update the counter
#         count_all.update(terms_all)
#         count_all.update(terms_stop)
#         count_all.update(terms_hash)
        count_all.update(terms_only)
        count_all.update(terms_single)
#         count_all.update(terms_bigram)
    # Print the first 5 most frequent words
    print(count_all.most_common(5))

[('hiring', 102), ('remote', 102), ('PHP', 102), ('Engineer', 102), ('gt', 102)]
